In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
plt.rc("font", family="Malgun Gothic") # 시각화 할때 한글이 볼수 있게 하겠다
plt.rc("axes", unicode_minus=False) # 마이너스 부호가 보일수 있게 하겠다
pd.options.display.max_columns = None
df = pd.read_csv('../main/수명주기붙힌거공백제거.csv',encoding='cp949')
df = df.replace(1254789.63,np.nan)
# df['설비투자효율'] = df['설비투자효율'].astype(float)
df = df[['회사명','거래소코드','회계년도','이자보상배율']]
df

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,201112,3.53
1,(주)CMG제약,58820,201212,-7.75
2,(주)CMG제약,58820,201312,0.70
3,(주)CMG제약,58820,201412,3.60
4,(주)CMG제약,58820,201512,-32.22
...,...,...,...,...
8743,흥아해운(주),3280,201512,1.02
8744,흥아해운(주),3280,201612,0.21
8745,흥아해운(주),3280,201712,-0.42
8746,흥아해운(주),3280,201812,-1.16


In [2]:
# # 전체 기록
# f_test_x = []
# count = 0
# company = df.iloc[0]['회사명']

# for i in range(0, len(df)):

#     # 회사명 다르면
#     if company != df.iloc[i]['회사명']:
#         # 회사명 기록하고
#         company = df.iloc[i]['회사명']
#         # 카운트 초기화하고
#         count = 0

#     #df.loc[i, 'count'] = count

#     # 이자보상배율이 0이면 탈출
#     if df.iloc[i]['이자보상배율'] == 0:
#         df.loc[i, '부실기업'] = 0
#         f_test_x.append(df.loc[i])
#         count = 0
#         continue

#     elif df.iloc[i]['이자보상배율'] < 1:
#          count += 1

#     else:
#         count = 0

#     # 이자보상배율에 따라 부실기업 여부 설정
#     if count > 1 and df.iloc[i]['이자보상배율'] >= 1:
#         df.loc[i, '부실기업'] = 0
#         pass

#     elif count > 1 and df.iloc[i]['이자보상배율'] < 1:
#         df.loc[i, '부실기업'] = 1

#     f_test_x.append(df.loc[i])

In [3]:
# df_test = pd.DataFrame(f_test_x)

In [4]:
# df_test

In [5]:
# df_test['부실기업'] = df_test['부실기업'].fillna(0)

In [6]:
# df_test['이보배2'] = df_test['이자보상배율'].shift().where(df_test['회사명']==(df_test['회사명']).shift())
# df_test

In [21]:
# 다음 연도의 이자보상배율을 저장
df['이보배2'] = df['이자보상배율'].shift(-1).where(df['회사명']==(df['회사명']).shift(-1))
df

,회사명,거래소코드,회계년도,이자보상배율,이보배2
0,(주)CMG제약,58820,201112,3.53,-7.75
1,(주)CMG제약,58820,201212,-7.75,0.70
2,(주)CMG제약,58820,201312,0.70,3.60
3,(주)CMG제약,58820,201412,3.60,-32.22
4,(주)CMG제약,58820,201512,-32.22,303.00
...,...,...,...,...,...
8743,흥아해운(주),3280,201512,1.02,0.21
8744,흥아해운(주),3280,201612,0.21,-0.42
8745,흥아해운(주),3280,201712,-0.42,-1.16
8746,흥아해운(주),3280,201812,-1.16,-0.55


In [22]:
# 부실징후기업의 경우 1, 아닌 경우 0을 기록
count = 0
company = df.iloc[0]['회사명']

for i in range(0, len(df)):
    if df.iloc[i]['이자보상배율'] < 1:
        df.loc[i, '부실기업'] = 1
    else:
        df.loc[i, '부실기업'] = 0

In [23]:
df['부실기업'] = df['부실기업'].fillna(0)

In [25]:
# t+1 이상의 기업들 중 살아난 기업 찾기
t_one = []
count = 0
count_for_f = 0
company = df.iloc[0]['회사명']

for i in range(0, len(df)):

    if company != df.iloc[i]['회사명']:
        company = df.iloc[i]['회사명']
        count = 0
        count_for_f = 0
        #continue #is it?

    # elif count > 0:
    #     continue

    if df.iloc[i]['부실기업'] == 1:
        count_for_f += 1
    #else:
        #count_for_f = 0 

    # if count_for_f > 2:
    #     continue

    elif df.iloc[i]['이자보상배율'] < 1 and df.iloc[i]['이보배2'] >= 1 and df.iloc[i]['부실기업']== 1 and count_for_f == 1:
        df.loc[i, '회생기업1'] = 1
        count += 1

In [26]:
df

,회사명,거래소코드,회계년도,이자보상배율,이보배2,부실기업
0,(주)CMG제약,58820,201112,3.53,-7.75,0.0
1,(주)CMG제약,58820,201212,-7.75,0.70,1.0
2,(주)CMG제약,58820,201312,0.70,3.60,1.0
3,(주)CMG제약,58820,201412,3.60,-32.22,0.0
4,(주)CMG제약,58820,201512,-32.22,303.00,1.0
...,...,...,...,...,...,...
8743,흥아해운(주),3280,201512,1.02,0.21,0.0
8744,흥아해운(주),3280,201612,0.21,-0.42,1.0
8745,흥아해운(주),3280,201712,-0.42,-1.16,1.0
8746,흥아해운(주),3280,201812,-1.16,-0.55,1.0


In [11]:
# t+2 이상의 기업들 중 살아난 기업 찾기

t_two = []
count = 0
company = df.iloc[0]['회사명']

for i in range(0, len(df)):

    if company != df.iloc[i]['회사명']:
        company = df.iloc[i]['회사명']
        count = 0
        #continue #is it?

    if df.iloc[i]['회생기업1'] == 1:
        count += 1
        #t_two.append(df.loc[i])
        continue

    # elif count == 1:
    #     if df.iloc[i]['부실기업'] == 0:
    #         #t_two.append(df.loc[i])
    #         count = 0

    # elif df.iloc[i]['부실기업'] == 1:
    #     continue

    elif count == 1:
        # 부실기업이 아니고 이자보상배율과 그 다음해 이자보상배율(이보배2)가 모두 1 이상으로 건실하다고 판단된 경우
        if df.iloc[i]['부실기업'] == 0 and df.iloc[i]['이자보상배율'] >= 1 and df.iloc[i]['이보배2'] >= 1:
            #if df.iloc[i]['이자보상배율'] < 1 and df.iloc[i]['이보배2'] >= 1:
                df.loc[i, "회생기업2"] = 1
        count += 1
        #t_two.append(df.loc[i])
        continue

    else:
        t_two.append(df.loc[i])

In [13]:
df3 = df.copy()
df3['회생기업1'] = df3['회생기업1'].fillna(0)
df3['회생기업2'] = df3['회생기업2'].fillna(0)

In [14]:
df3['회생기업1'].value_counts()

0.0    8375
1.0     373
Name: 회생기업1, dtype: int64

In [15]:
df3['회생기업2'].value_counts()

0.0    8519
1.0     229
Name: 회생기업2, dtype: int64

In [16]:
df3.to_csv("t회생시험찐.csv", encoding="CP949",index=False)

In [12]:
df['회생기업1'] = df['회생기업1'].shift().where(df['회사명']==(df['회사명']).shift())
df['회생기업2'] = df['회생기업2'].shift().where(df['회사명']==(df['회사명']).shift())

In [36]:
no = []
safe = []
for i in range(0, len(df)-1):
    if df.iloc[i]['회사명'] == df.iloc[i-1]['회사명'] and df.iloc[i]['회사명'] == df.iloc[i+1]['회사명']:
        if df.iloc[i-1]['이자보상배율'] < 1 and df.iloc[i]['이자보상배율'] >= 1 and df.iloc[i+1]['이자보상배율'] >= 1:
                df.loc[i+1, '회생여부'] = 1
                safe.append()

In [38]:
# df_test['회생여부'] = df_test['회생여부'].fillna(0)

In [39]:
df.to_csv('t+2회생여부.csv',encoding='cp949',index=False)

In [68]:
# df_test["회생여부"] = df_test["회생여부"].shift().where(df['회사명']==(df['회사명']).shift())

In [69]:
# t+2 이상의 기업들 중 살아난 기업 찾기
# 숫자 2 이상으로 조절해서 나가시면 됩니다
# 밑에 파일명 바꾸는 거 조심하시구요

count = 0
company = df_test.iloc[0]['회사명']

for i in range(0, len(df_test)):

    if company != df_test.iloc[i]['회사명']:
        company = df_test.iloc[i]['회사명']
        count = 0
        continue # is it?

    if df_test.iloc[i]['부실기업'] == 1:
        count += 1
        continue

    if count == 1:
        if df_test.iloc[i]['부실기업'] == 0:
            count = 0

    elif count >= 2:
        # 부실기업이 아니고 이자보상배율과 그 다음해 이자보상배율(이보배2)가 모두 1 이상으로 건실하다고 판단된 경우
        if df_test.iloc[i]['부실기업'] == 0 and df_test.iloc[i]['이자보상배율'] >= 1 and df_test.iloc[i]['이보배2'] >= 1:
            #if df_test.iloc[i]['이자보상배율'] < 1 and df_test.iloc[i]['이보배2'] >= 1:
                df_test.loc[i, "회생기업2"] = 1
        count = 0

In [70]:
df_test.drop_duplicates('회사명')

,회사명,거래소코드,회계년도,이자보상배율,부실기업,이보배2,회생여부
0,(주)CMG제약,58820,11-Dec,3.53,0.0,NaN,NaN
9,(주)DB하이텍,990,11-Dec,-0.75,0.0,NaN,NaN
18,(주)ES큐브,50120,11-Dec,-245.99,0.0,NaN,NaN
27,(주)HJ중공업,97230,11-Dec,0.55,0.0,NaN,NaN
36,(주)MH에탄올,23150,11-Dec,0.12,0.0,NaN,NaN
...,...,...,...,...,...,...,...
8790,화천기계(주),10660,11-Dec,68.14,0.0,NaN,NaN
8799,휴림네트웍스(주),192410,14-Dec,3.38,0.0,NaN,NaN
8805,휴림로봇(주),90710,11-Dec,2.96,0.0,NaN,NaN
8814,흥구석유(주),24060,11-Dec,10.35,0.0,NaN,NaN


In [71]:
df_test[9:20]

,회사명,거래소코드,회계년도,이자보상배율,부실기업,이보배2,회생여부
9,(주)DB하이텍,990,11-Dec,-0.75,0.0,NaN,NaN
10,(주)DB하이텍,990,12-Dec,-0.31,1.0,-0.75,NaN
11,(주)DB하이텍,990,13-Dec,-0.16,1.0,-0.31,NaN
12,(주)DB하이텍,990,14-Dec,0.67,1.0,-0.16,NaN
13,(주)DB하이텍,990,15-Dec,2.50,0.0,0.67,NaN
14,(주)DB하이텍,990,16-Dec,4.41,0.0,2.50,NaN
15,(주)DB하이텍,990,17-Dec,6.05,0.0,4.41,NaN
16,(주)DB하이텍,990,18-Dec,6.77,0.0,6.05,NaN
17,(주)DB하이텍,990,19-Dec,13.44,0.0,6.77,NaN
18,(주)ES큐브,50120,11-Dec,-245.99,0.0,NaN,NaN


In [72]:
df_test[df_test['회생여부']==1]

,회사명,거래소코드,회계년도,이자보상배율,부실기업,이보배2,회생여부
